ref: https://github.com/onnx/tutorials/blob/master/tutorials/PytorchOnnxExport.ipynb

# Save pre-trained model to ONNX

In [1]:
from torch.autograd import Variable
import torch.onnx
import torchvision

# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = Variable(torch.randn(1, 3, 224, 224))
# Obtain your model, it can be also constructed in your script explicitly
model = torchvision.models.alexnet(pretrained=True)
# Invoke export
torch.onnx.export(model, dummy_input, "alexnet.onnx")

# Load model from ONNX

In [2]:
import onnx

# Load the ONNX model
model = onnx.load("alexnet.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %1[FLOAT, 1x3x224x224]
) initializers (
  %2[FLOAT, 64x3x11x11]
  %3[FLOAT, 64]
  %4[FLOAT, 192x64x5x5]
  %5[FLOAT, 192]
  %6[FLOAT, 384x192x3x3]
  %7[FLOAT, 384]
  %8[FLOAT, 256x384x3x3]
  %9[FLOAT, 256]
  %10[FLOAT, 256x256x3x3]
  %11[FLOAT, 256]
  %12[FLOAT, 4096x9216]
  %13[FLOAT, 4096]
  %14[FLOAT, 4096x4096]
  %15[FLOAT, 4096]
  %16[FLOAT, 1000x4096]
  %17[FLOAT, 1000]
) {
  %19 = Conv[dilations = [1, 1], group = 1, kernel_shape = [11, 11], pads = [2, 2, 2, 2], strides = [4, 4]](%1, %2)
  %20 = Add[axis = 1, broadcast = 1](%19, %3)
  %21 = Relu(%20)
  %22 = MaxPool[kernel_shape = [3, 3], pads = [0, 0], strides = [2, 2]](%21)
  %24 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%22, %4)
  %25 = Add[axis = 1, broadcast = 1](%24, %5)
  %26 = Relu(%25)
  %27 = MaxPool[kernel_shape = [3, 3], pads = [0, 0], strides = [2, 2]](%26)
  %29 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 

# Inference with your prefered toolkit

In [3]:
from PIL import Image 
import numpy as np
imgfile = './data/dog.jpg'
img = Image.open(imgfile).resize( (224, 224) )
img_arr = np.array(img)
img_arr = np.expand_dims(img_arr, -1)
img_arr = np.transpose(img_arr, (3,2,0,1))
img_arr.shape

(1, 3, 224, 224)

In [5]:
import onnx_caffe2.backend as backend

rep = backend.prepare(model, device="CUDA:0") # or "CPU"
outputs = rep.run(img_arr.astype(np.float32))

No handlers could be found for logger "caffe2.python.workspace"


In [6]:
print(outputs[0].shape)

(1, 1000)
